In [ ]:
from envs.test_env import Electric_Car
from models.baseline import BaselineModel
from models.baseline2 import BaselineModel2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def print_func(states, rewards, infos):
    batteries = np.array(states)[:, 0]
    prices = np.array(states)[:, 1]
    for i in range(450):
        print('t=%d, battery=%.2f, price=%.2f, action=%d, rewards=%.2f, hour=%d'%(i, batteries[i], prices[i], infos[i], rewards[i], states[i][2]))
    
    
    #print('total reward = ', sum(rewards))
    
    '''for i in range(len(battery)):
        if actions[i] != 25:
            hour = i%24
            print('t=%d, hour=%d battery=%.2f, price=%.2f, action=%d, rewards=%.2f'%(i, hour, battery[i], prices[i], actions[i], rewards[i]))
    '''
    
    return

def plot_func(states, rewards, infos):
    batteries = np.array(states)[:, 0]
    prices = np.array(states)[:, 1]
    print(batteries)
    print(prices)
    x = range(len(states))
    charging = np.insert(np.diff(batteries), 0, 0)
    cum_rewards = np.cumsum(np.array(rewards))
    
    fig, axis = plt.subplots(2,3, figsize=(12,8))
    for ax, val, name in zip(axis.flat, 
                             [batteries, charging, prices, infos, rewards, cum_rewards],
                               ['Battery', 'Charging', 'Prices', 'Actions', 'Rewards', 'Cumulative rewards']):
        ax.scatter(x, val, s=1)
        ax.set_title(name)
    plt.show()
    
    
    plt.figure(figsize=(12, 6))
    col = np.where(np.array(infos)>0,'red',np.where(np.array(infos)<0, 'green', '#FF000000'))
    plt.ylim(0, 300)
    plt.xlim(0, 450)
    plt.scatter(x, prices, c = col,s = 40, zorder=1)
    plt.plot(x, prices, zorder =0, alpha=0.8)
    upper_line = np.full(shape=len(states), fill_value=104.67949999999989, dtype=np.float64)
    lower_line = np.full(shape=len(states), fill_value=39.99, dtype=np.float64)
    plt.plot(x, upper_line, c= "green", alpha=0.5)
    plt.plot(x, lower_line, c= "red", alpha=0.5, zorder=1)
    plt.ylabel('Price in Euro')
    plt.xlabel('Time')
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.plot(x, cum_rewards)
    plt.xlabel('Time')
    plt.ylabel('Cumulative Reward in Euro')

    return


In [ ]:
min_thres = [0.05, 0.10, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45]
max_thres = [0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]


Rewards = []
for Min in min_thres:
    for Max in max_thres:
            env = Electric_Car('data/train.xlsx')
            model = BaselineModel(env, Min, Max)
            states, rewards, terminated, truncated, infos  = learn(env, model)
            Rewards.append([sum(rewards), Min, Max])

            #print_func(battery, prices, actions, rewards)
            #plot_func(battery, prices, actions, rewards)
            
    max_idx = np.argmax(np.array(Rewards)[:,0])
    print(Rewards[max_idx])

In [ ]:
env = Electric_Car('data/train.xlsx')
model = BaselineModel(env, 0.1, 0.95)
states, rewards, terminated, truncated, infos = learn(env, model)
plot_func(states, rewards, infos)


In [ ]:
env = Electric_Car('data/train.xlsx')
model = BaselineModel2(env)
states2, rewards2, terminated2, truncated2, infos2 = learn(env, model)
print(sum(rewards2))

In [ ]:
plot_func(states2, rewards2, infos2)

In [ ]:
plt.figure(figsize=(12, 6))
battery = np.array(states2)[:, 0]
x = range(len(battery))
cum_rewards = np.cumsum(np.array(rewards))
cum_rewards2 = np.cumsum(np.array(rewards2))
plt.plot(x, cum_rewards - cum_rewards2, label = 'Profit Baseline Model vs 7 AM buying')
#plt.plot(x, cum_rewards2, label = 'Buying at 7AM')
plt.legend(fontsize = 12)
plt.xlabel('Time')
plt.ylabel('Profit in Euro')